
# Om å finne illustrasjoner i bøker fra NB digital

Fra en oversikt over alle illustrasjoner i NB-digital (ca. 480 000 bøker) kan vi studere selve illustrasjonene. Hva som kan vises er avhengig av tilgangsnivå, og hvor vi er, på jobb på NB eller på privatmaskin. 

Det frie materialet kan nås fra hvor som helst, mens det som har restriksjoner også er begrenset innad i NB. For begrenset materiale er det kun mulig å få ut deler av en illustrasjon, men det er mulig å mekke det til så man får sett alt. Foreløpig er ikke det gjort. 

### Litt startkode

In [13]:
import dhlab.module_update as mu
mu.update("nbpictures", silent = True)
from nbpictures import show_illustrations_urn, urns_from_super, load_picture, get_urls_from_illustration_data, get_illustration_data_from_book
import nbpictures as nbp
from IPython.display import HTML, Markdown, display
from skimage import data, io, filters, feature, color
import pandas as pd
import matplotlib.pyplot as plt
import os

Hent ut noen URN-er, eller bruk URN-er funnet annet sted

In [15]:
import imp

In [16]:
imp.reload(nbp)

<module 'nbpictures' from 'C:\\Users\\yoons\\Documents\\GitHub\\illustrations_bokhylla\\nbpictures.py'>

In [11]:
small_scale = 0.59
large_scale = 1.58

In [19]:
def get_urls_from_illustration_data(illus, part = True, scale = None, cuts = True, cutout = None, delta = 0):
    """part sets size of output of page, if part is True it returns a cut out of image
    using position data in illus is a dictionary of with entries and values like this: 
    {'height': 270, 'hpos': 251, 'page': 'digibok_2017081626006_0018', 'resolution': 400, 'vpos': 791, 'width': 373} 
    the variable cuts, if true allows cropping of image - restricted images must not go over 1024 x 1024 pixels
    
    The variable cutout, if different from zero, picks out a part of the picture, specifically made for
    finding the signature part of illustrations.
    """
    
    urn = "URN:NBN:no-nb_" + illus['page']
    
    if scale == None:
        if int(illus['resolution']) >= 300 or int(illus['resolution']) < 100:
            scale = large_scale
        else:
            scale = small_scale
            
    height = int(illus['height']) + 2*delta
    width = int(illus['width']) + 2*delta
    vpos = int(illus['vpos']) - delta
    hpos = int(illus['hpos']) - delta
    
    # check cutout, then that value will take precedence
    if cutout != None:
        (widthc, heightc) = cutout
        vpos = vpos + width - widthc
        hpos = hpos + height - heightc
        width = widthc
        height = heightc
        
    if cuts != False:
        if width * scale > 1024:
            width = int(1024/scale)
        if height * scale > 1024:
            height = int(1024/scale)
            

    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(width * scale), 
            height = int(height * scale), 
            vpos = int(vpos * scale), 
            hpos = int(hpos * scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

In [51]:
#!/usr/bin/env python
# coding: utf-8

# In[8]:


import sqlite3
import os
import tarfile
import shutil
import dhlab.nbtokenizer as tok


# In[4]:


def alto_extract(altofile, to_path = '.'):
    """
    Pakk ut tarfil til ramdisk og returnerer mappen filene ligger i
    """


    tf = tarfile.open(altofile, 'r')
    # lag mappe på disk med navnet foran det som står foran .tar
    filename = os.path.basename(altofile)
    dname = filename.split('.tar')[0]
    ndir = os.path.join(to_path, dname )
    os.mkdir(ndir)
    # Pakk ut alt til mappen
    tf.extractall(ndir)
    tf.close()
    return ndir


def process_alto(ndir):
    """Hent ut alle ordene i alto-filen, og legg til paragraf og sidenummer. Mappen ndir peker til mappen der alto-xml-filene ligger"""
    
    import xml.etree.ElementTree as ET
    import shutil
    
    # XML-filene ligger i mappen ndir, så gå gjennom med os.walk()
    # Alle filene blir liggende i variabelen f
    r,d,f = next(os.walk(ndir))
    
    # hent sidene i teksten og legg dem i variabelen pages
    # skip metadatafilene - tekstene har sidenummer representert som 4-sifrede nummer, f.eks. 0014
    pages = []
    
    for page in f:
        pag = page.split('.xml')[0].split('_')[-1]
        try:
            int(pag)
            pages.append((page, int(pag)))
        except:
            True
        
    # Gå gjennom side for side og hent ut teksten. Delte ord blir lagt i variabelen hyph,
    # teksten i text. Alle ord får et sekvensnummer relativt til boka det står i, samtidig
    # som alle avsnitt blir nummerert fortløpende
    
    para_num = 1
    word_num = 1

    text = []
    hyph = []

    hyp1 = ""
    hyp2 = ""
    
    # sorter variabelen pages på sidenummer, andre ledd i tuplet
    for page in sorted(pages, key=lambda x: x[1]):
        page_file = os.path.join(r, page[0])
        page_num = page[1]
        
        # parse XML-fila og få tak i rotelementet root
        tree = ET.parse(page_file)
        root = tree.getroot()
        
        # Gå gjennom XML-strukturen via TextBlock, som er avsnittselementet
        for paragraph in root.findall(".//TextBlock"):
            
            # Finn alle ordene i avsnittet, som attributter til elementet String,
            # og sjekk om det foreligger en orddeling -
            # i så fall ligger hele ordet i attributtet SUBS_CONTENT, mens første ledd av orddelingen
            # ligger i CONTENT. Om det ikke er noen orddeling ligger ordet i attributtet CONTENT.
            # Burde fungere også med orddelinger over sideskift
            
            # Ordet lagres sammen med sekvensnummeret og sekvensnummeret for avsnittet står i,
            # i tillegg til sidenummeret, som kan være greit for oppslag i bokhylla, i forbindelse
            # med generering av konkordanser.
            
            for string in paragraph.findall(".//String"):
                if 'SUBS_TYPE' in string.attrib:
                    if string.attrib['SUBS_TYPE'] == "HypPart1":
                        #tokens = tok.tokenize(string.attrib['SUBS_CONTENT'])
                        tokens = [string.attrib['SUBS_CONTENT']]
                        for token in tokens:
                            text.append((token, word_num, para_num, page_num))
                            word_num += 1
                    elif string.attrib['SUBS_TYPE'] == "HypPart2":
                        hyp2 = string.attrib['CONTENT']
                        hyph.append((hyp1, hyp2))
                else:
                    #tokens = tok.tokenize(string.attrib['CONTENT'])
                    tokens = [string.attrib['CONTENT']]
                    for token in tokens:
                        text.append((token, word_num, para_num, page_num))
                        word_num += 1
            para_num += 1
    # returner teksten som en sekvens av tupler, sammen med orddelingene, også som en sekvens av tupler
    return text #, hyph

In [65]:
def extract_image_alto(ndir):
    """Hent ut alle bildene i alto-filen. Mappen ndir peker til mappen der tarfilene ligger"""
    
    import xml.etree.ElementTree as ET
    import shutil
    
    ns = {'mix': "http://www.loc.gov/mix/"} 
    # XML-filene ligger i mappen ndir, så gå gjennom med os.walk()
    # Alle filene blir liggende i variabelen f
    
    r,d,f = next(os.walk(ndir))
    
    
    # hent oppløsningen fra metsfil
    metsfile = [fmets for fmets in f if fmets.endswith('mets.xml')]
    if metsfile != []:
        metsfile = os.path.join(r, metsfile[0])
        tree = ET.parse(metsfile)
        root = tree.getroot()
    
        resolutions = {element.text for element in root.findall(".//mix:XphysScanResolution", ns)}
        resolution = list(resolutions)[0]
    else:
        resolution = 100 
    
    pages = []
    
    for page in f:
        pag = page.split('.xml')[0].split('_')
        try:
        #    int(pag[2])
            pages.append((page, pag[2]))
            
        except:
            #print(pag)
            True
    # print(pages)
    # alle avsnitt blir nummerert fortløpende
    
    illustrations = []
    # sorter variabelen pages på sidenummer, andre ledd i tuplet
    for page in sorted(pages, key=lambda x: x[1]):
        page_file = os.path.join(r, page[0])
        page_num = page[1]
        file_identifier = page[0].split('.')[0] # remove .xml
        #print(page[0], page[1])
        # parse XML-fila og få tak i rotelementet root
        tree = ET.parse(page_file)
        root = tree.getroot()
        #print(root)
        # Gå gjennom XML-strukturen via TextBlock, som er avsnittselementet
        for paragraph in root.findall(".//*[@TYPE='Illustration']"):
            illustrations.append(
                {'page': file_identifier, 
                 'pagenum':page_num,
                 'resolution': resolution,
                 'hpos': paragraph.attrib["HPOS"],
                 'vpos':paragraph.attrib["VPOS"],
                 'width':paragraph.attrib["WIDTH"],
                 'height':paragraph.attrib["HEIGHT"],
                 'type': 'Illustration'
                 }
            )
        for paragraph in root.findall(".//*[@TYPE='GraphicalElement']"):
            illustrations.append(
                {'page': file_identifier,
                 'pagenum':page_num,
                 'resolution': resolution,
                 'hpos': paragraph.attrib["HPOS"],
                 'vpos':paragraph.attrib["VPOS"],
                 'width':paragraph.attrib["WIDTH"],
                 'height':paragraph.attrib["HEIGHT"],
                 'type': 'GraphicalElement'
                 }
            )
        
    return illustrations


In [22]:
from nbpictures import iiif_manifest

In [107]:
def display_books(books, width = 100):
    """A dictionary of urns - urls is displayed """
    import sys
    html_wrapper = lambda x: """<style>
    img {{
        width:{mxw}px;
        height:auto;
        max-widht:}}
    </style>
    <body>{body}</body>""".format(body = x, mxw = width)

    div_wrapper = lambda x: """<div>{div_content}</div>""".format(div_content = x)
    book_divs = ""
    for u in books:
        try:
            mf = iiif_manifest(u)
            #print('iif')
            thumbnail = "<img src = '{thumbnail}'></img>".format(thumbnail = mf['thumbnail']['@id'])
            #print('thumbnail')
            metainfo =  '\n'.join(["<b>{label}</b>{val}".format(label = x['label'], val = x['value']) for x in mf['metadata']])        
            imgs = '\n'.join(["<img src='{img_http}'></img>".format(img_http = pic_url) for pic_url in books[u]])
            #print('images')
            book_divs += div_wrapper(thumbnail + metainfo + imgs)
            #print('book_divs')
        except KeyboardInterrupt:
            break;
        except:
            print("Unexpected error:", sys.exc_info()[0])
            print("Problemer med: ", u, e)
            True
    return html_wrapper(book_divs)

In [ ]:
def markdown_books(books, width = 100):
    """A dictionary of urns - urls is displayed """
    
    markdown_wrapper = lambda x: """
    <body>{body}</body>""".format(body = x)

    div_wrapper = lambda x: """<div>{div_content}</div>""".format(div_content = x)
    
    book_divs = ""
    for u in books:
        try:
            mf = iiif_manifest(u)
            thumbnail = "<h3>Forside</h3> <img src='{thumbnail}'></img> <h3>Metadata</h3>".format(thumbnail = mf['thumbnail']['@id'])
            metainfo =  '\n'.join([" <b>{label}</b> {val}".format(label = x['label'], val = x['value']) for x in mf['metadata']])        
            imgs = '\n'.join(["<img style='float:right' src='{img_http}' width = {width}></img>".format(img_http = pic_url, width = width) for pic_url in books[u]])
            book_divs += div_wrapper(thumbnail + metainfo + imgs)
        except:
            True
    return book_divs

In [41]:
def markdown_illustrations(books, width = 100):
    """A list of resolver urls is displayed """
    import re
    
    markdown_wrapper = lambda x: """
    <body>{body}</body>""".format(body = x)

    div_wrapper = lambda x: """<div>{div_content}</div>""".format(div_content = x)
    book_divs = ""
    for u in books:
        urn = re.findall("URN:NBN:no-nb_digibok_[0-9]{13}", u)[0]
        mf = iiif_manifest(urn)
        thumbnail = "<h3>Forside</h3> <img src='{thumbnail}'></img> <h3>Metadata</h3>".format(thumbnail = mf['thumbnail']['@id'])
        metainfo =  '\n'.join([" <b>{label}</b> {val}".format(label = x['label'], val = x['value']) for x in mf['metadata']])        
        imgs = "<img style='float:right' src='{img_http}' width = {width}></img>".format(img_http = u, width = width) 
        book_divs += div_wrapper(thumbnail + metainfo + imgs)
    return book_divs

In [111]:
def dict_sample( a_dict, n = 10):
    from random import sample
    keys = list(a_dict.keys())
    sample_dict = {u:a_dict[u] for u in sample(keys, min(n, len(keys)))}
    return sample_dict

In [45]:
r, d, f = next(os.walk('../../skillingsviser-master/skillingsviser_alto/'))


In [80]:
illustrations = {"URN:NBN:no-nb_" + '_'.join(alto.split('_')[:2]):extract_image_alto(os.path.join(r,alto)) for alto in d}

#illustrations = [x for y in illustrations for x in y]

#illustrations

In [81]:
illustrations

{'URN:NBN:no-nb_digibok_2017103026032': [],
 'URN:NBN:no-nb_digibok_2018021528001': [{'page': 'digibok_2018021528001_I1',
   'pagenum': 'I1',
   'resolution': '400',
   'hpos': '67',
   'vpos': '168',
   'width': '138',
   'height': '177',
   'type': 'Illustration'}],
 'URN:NBN:no-nb_digibok_2018021528002': [{'page': 'digibok_2018021528002_C1',
   'pagenum': 'C1',
   'resolution': '400',
   'hpos': '72',
   'vpos': '1036',
   'width': '1161',
   'height': '778',
   'type': 'Illustration'},
  {'page': 'digibok_2018021528002_I3',
   'pagenum': 'I3',
   'resolution': '400',
   'hpos': '104',
   'vpos': '277',
   'width': '136',
   'height': '159',
   'type': 'Illustration'}],
 'URN:NBN:no-nb_digibok_2018021528003': [{'page': 'digibok_2018021528003_I1',
   'pagenum': 'I1',
   'resolution': '400',
   'hpos': '96',
   'vpos': '255',
   'width': '112',
   'height': '133',
   'type': 'Illustration'}],
 'URN:NBN:no-nb_digibok_2018021528004': [{'page': 'digibok_2018021528004_C1',
   'pagenum': '

Konverter bøker til illustrasjonsdata, formatet er en dict av

```{urn: [url, url, ..],
urn2: [url, url, ..],
...
}```

In [128]:
skillingsbilder = {urn:[get_urls_from_illustration_data(illus, part=1500) for illus in illustrations[urn]] for urn in illustrations}

In [130]:
Markdown('\n\n'.join([ u + "\n\n" + '\n\n'.join(skillingsbilder[u]) for u in skillingsbilder]))

URN:NBN:no-nb_digibok_2017103026032



URN:NBN:no-nb_digibok_2018021528001

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528001_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528002

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528002_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528002_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528003

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528003_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528004

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528004_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528006_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528006_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528008

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528008_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528012



URN:NBN:no-nb_digibok_2018021528013

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528013_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528013_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528014

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528014_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528014_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528014_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528015_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528016_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528017_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528019

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528019_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528020

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528020_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018021528021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628002

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628002_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628003



URN:NBN:no-nb_digibok_2018022628006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628006_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628008

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628008_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628010_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628010_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628011_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628011_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628012

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628012_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628013

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628013_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628013_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628015_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628015_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628015_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628016_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628017_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628017_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628019



URN:NBN:no-nb_digibok_2018022628020

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628020_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628020_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628021_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628021_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628022

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628022_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628023

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628023_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628023_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628024

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628024_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628025

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628025_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628025_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628027

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628027_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628027_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628029

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628029_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628031

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628031_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628032

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628032_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628032_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628032_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628033

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628033_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628034

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628034_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628035

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628035_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628035_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628036

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628036_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628037

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628037_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628037_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628038

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628038_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018022628039



URN:NBN:no-nb_digibok_2018102326002

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326002_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326002_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326002_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326003

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326003_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326003_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326004

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326004_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326006_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326006_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326006_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326007

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326007_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326007_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326008

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326008_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326008_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326009

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326009_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326010_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326010_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326011_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326013

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326013_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326013_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326013_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326014

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326014_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326014_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326014_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326015_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326016_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326016_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326016_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326017_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326019

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326019_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326019_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326020

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326020_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326020_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326021_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326022

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326022_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326022_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326026

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326026_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326028

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326028_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326028_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326029



URN:NBN:no-nb_digibok_2018102326030

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326030_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326032

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326032_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326033

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326033_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326033_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326034

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326034_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326035

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326035_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326036

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326036_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326037

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326037_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326037_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326039

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326039_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326039_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326040

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326040_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326040_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326041

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326041_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326041_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326042

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326042_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326042_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326043

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326043_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326044

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326044_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326045

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326045_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326045_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326046

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326046_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326046_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326046_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326048

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326048_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326049

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326049_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326049_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326050

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326050_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326051

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326051_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326052

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326053

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326053_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326054

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326054_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326054_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326054_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326057

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326057_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326058

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326058_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326059



URN:NBN:no-nb_digibok_2018102326061

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326061_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326061_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326061_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326062

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326062_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326063

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326063_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326064

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018102326065

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326065_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326065_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826003

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826003_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826004

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826004_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826004_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826005

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826005_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826006_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826006_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826007

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826007_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826007_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826009

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826009_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826010_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826010_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826011_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826011_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826012

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826012_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826015_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826016_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826017_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826017_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826018

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826018_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826019

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826019_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826019_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826020



URN:NBN:no-nb_digibok_2018112826021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826024

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826024_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826024_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826025

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826025_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826025_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826027



URN:NBN:no-nb_digibok_2018112826028

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826028_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826028_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826028_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826029

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826029_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826029_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826030

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826030_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826030_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826033

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826033_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826033_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826033_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826034

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826034_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826036

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826036_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826036_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826037

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826037_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826038

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826039

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826039_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826043

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826043_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826043_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826044



URN:NBN:no-nb_digibok_2018112826045



URN:NBN:no-nb_digibok_2018112826046

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826047



URN:NBN:no-nb_digibok_2018112826048

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826048_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826048_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826049

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826049_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826050



URN:NBN:no-nb_digibok_2018112826052

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826052_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826054

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826054_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826054_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826054_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826057

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826058

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826058_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826058_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826058_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826060

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826060_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826063

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826063_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826063_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826064

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826064_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826064_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2018112826065

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826065_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826065_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726008



URN:NBN:no-nb_digibok_2019030726009

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726009_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726010_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726010_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726010_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726011_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726011_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726011_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726012

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726012_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726012_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726013



URN:NBN:no-nb_digibok_2019030726014

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726014_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726014_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726015_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726015_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726016_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726016_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726016_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726017_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726018



URN:NBN:no-nb_digibok_2019030726021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726021_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726022

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726022_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726023

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726023_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726023_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726024

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726025

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726025_0002/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726025_0003/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726025_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726026

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726026_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726026_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726027

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726027_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726028

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726028_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726029

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726029_0001/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726029_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726030



URN:NBN:no-nb_digibok_2019030726031

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726031_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726032

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726032_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726033

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726033_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726034

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726034_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726034_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726035

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726035_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726036

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726036_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726037

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726037_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726037_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726038

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726039

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726039_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726040

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726040_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726040_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726040_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726041



URN:NBN:no-nb_digibok_2019030726042

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726042_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726042_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726043



URN:NBN:no-nb_digibok_2019030726044

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726044_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726044_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726045

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726045_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726045_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726046

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726046_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019030726047

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726047_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426002

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426002_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426002_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426003

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426003_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426004

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426004_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426004_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426005

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426005_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426005_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426006_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426006_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426007

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426007_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426009

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426009_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426010_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426011_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426011_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426012



URN:NBN:no-nb_digibok_2019031426013

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426013_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426013_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426014

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426014_0002/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426014_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0001/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0002/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0002/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0004/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426016_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426016_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426017_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426017_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426018

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426018_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426018_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426019

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426019_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426019_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426020

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426020_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426020_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019031426022

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426022_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326001

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326001_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326001_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326002

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326002_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326002_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326002_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326003

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326003_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326003_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326004

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326004_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326004_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326005

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326005_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326005_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326006_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326006_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326007

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326007_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326007_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326008

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326008_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326008_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326009

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326009_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326009_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326010_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326010_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326011_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326011_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326013

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326014

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326014_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326014_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326015_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326015_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326017_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326017_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326018

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326018_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326018_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326019

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326019_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326019_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326020

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326021_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326022

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326022_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326022_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326023

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326023_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326023_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326024

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326024_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326024_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326024_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326025

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326025_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326025_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326025_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326026

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326026_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326026_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326027

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326027_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326029

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326029_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326029_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326030

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326030_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326031

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326031_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326031_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326038

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326038_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326038_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326039

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326039_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326040

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326040_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326040_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061326041

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326041_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926001

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926001_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926002

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926002_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926002_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926002_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926003

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926003_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926004

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926004_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926004_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926005

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926005_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926005_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926006_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926006_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926007

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926007_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926007_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926008

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926008_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926010_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926010_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926011_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926011_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926011_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926012

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926012_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926012_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926013

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926013_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926013_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926014

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926014_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926014_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926015_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926016_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926016_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926016_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926017_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926018

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926018_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926018_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926018_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019061926019

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926019_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926002

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926002_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926003

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926003_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926003_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926004

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926004_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926005

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926005_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926005_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926006

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926006_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926007

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926007_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926007_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926008

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926008_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926008_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926008_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926009



URN:NBN:no-nb_digibok_2019112926010

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926010_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926010_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926011

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926011_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926012

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926012_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926013

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926013_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926014

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926015

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926016

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926017

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926017_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926018

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926018_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926019

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926019_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926019_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926020

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926020_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926021

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926021_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926021_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926022

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926022_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926022_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926022_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926023

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926023_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926023_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926023_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926024

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926024_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926024_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926025



URN:NBN:no-nb_digibok_2019112926026

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926026_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926026_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926027

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926028

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926028_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926029

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926029_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926029_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926030

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926030_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926030_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926031

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926031_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926031_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926032

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926032_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926032_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926033

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926034

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926034_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926034_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926034_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926035

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_I1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_I3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926036

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926036_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926036_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926037

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926037_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926038

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926038_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926038_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926039

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926039_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926039_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926040

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926040_C3/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926040_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926041

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926041_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926042

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926042_I1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926043

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926043_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926044

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926044_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926045

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926045_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926045_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926046

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926046_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926046_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926047

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926047_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926047_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926048

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926048_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926048_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926048_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926049

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926049_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926049_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926050

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926050_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926050_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926051

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926051_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926051_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926052

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926052_C1/full/0,1500/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926052_C3/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926053

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926053_C1/full/0,1500/0/native.jpg

URN:NBN:no-nb_digibok_2019112926054

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926054_I1/full/0,1500/0/native.jpg

Lagrer skillingsvisene til fil

In [131]:
import json
json.dump(skillingsbilder, open('skillingsbilder.json','w', encoding = 'utf-8'))

Se på bilder

In [137]:
html_bilder = display_books(dict_sample(skillingsbilder), 400)

In [138]:
HTML(html_bilder)